#### 라이브러리 import

In [ ]:
import os
from google import genai
from typing import List, Dict
from dotenv import load_dotenv

load_dotenv()

GEMINII_API_KEY = os.getenv("GEMINI_API_KEY")

In [ ]:
def generation(client, llm_model_params, query:str, search_results: List[Dict]):
    """
    Retrieval 결과와 Query를 바탕으로 답변을 생성

    Args:
        query (str): 사용자 질문
        search_results (List[Dict]): Qdrant 검색 결과 (payload) 포함
    """

    # Context 조합
    context_list = []

    for rank, result in enumerate(search_results):
        context_text = result.payload.get('text', 'no text')
        context_list.append(f"[{rank+1} {context_text}]")

    context = "\n--\n".join(context_list)

    # 프롬프팅 전략
    # System Instruction: Role Prompting
    system_prompt = (
        "당신은 주어진 Context에 기반하여 사용자의 질문에 정확하고 간결하게 답변하는 전문가입니다."
        "답변은 오직 Context에 있는 정보만 사용해야 합니다. 만약 Context에서 질문에 대한 답을 찾을 수 없다면, '관련 정보를 찾을 수 없음'이라고만 답변하세요."
    )

    # User Prompt: CoT & 출처 명시 형식 요구
    user_prompt = (
        "주어진 Context를 분석하여 다음 Query에 답변하는 단계를 '추론' 섹션에 작성하고, 최종 답변을 '답변' 섹션에 작성하세요. 최종 답변에는 참조한 문맥의 번호([N])를 반드시 명시해야 합니다."
        f"문맥(Context):\n{context}\n"
        f"질문(Query):\n{query}\n"
        f"출력 형식: \n"
        f"추론: [답변 도출 과정]\n"
        f"답변: [최종 답변 내용(반드시 출처 번호 명시)]"
    )

    try:
        response = client.models.generate_content(
            model=llm_model_params['model'],
            contents=[
                {'role': 'user', 'parts': [{'text': user_prompt}]}
            ],
            config={
                'system_instruction': system_prompt,
                'temperature': llm_model_params['temperature'],
                'max_output_tokens': llm_model_params['max_tokens']
            }
        )
        return response.text
    
    except Exception as e:
        return f"Gemini API 답변 생성 중 오류 발생: {e}"


In [ ]:
# Gemini API 클라이언트 설정
client = genai.Client(api_key=GEMINII_API_KEY)

LLM_MODEL_PARAMS = {
    'temperature':0.2,
    'max_tokens':12,
    'model':'gemini-2.5-flash'
}